In [1]:
import copy
import sys
import pathlib as pl

home = pl.Path.home()
omph_path = home / 'work' / 'Omphalos'
tope_path = home / 'work' / 'topepan'

sys.path.append(str(omph_path))
sys.path.append(str(tope_path))
import omphalos
import topepan as tp
from topepan import plot as tpp
from omphalos import file_methods as fm
from coeus import helper as hp
import xarray as xr

In [53]:
# Read in data for dict.
pickle = 'cr_08-03-23.pkl'
data_path = home / 'work' / 'data' / 'hydrothermal_alteration' / pickle
raw = fm.unpickle(data_path)

set1, errors1 = hp.filter_errors(raw)
set1 = hp.fix_smalls(set1, 'volume')

#raw = fm.unpickle('/home/angus/work/data/hydrothermal_alteration/700_15deg.pkl')

#set2, errors2 = hp.filter_errors(raw)
#set2 = hp.fix_smalls(set2, 'volume')

datasets = [set1]

Returned 29 files without errors out of a total possible 29.
0 files had errors.
0 files had unhandled errors.
File failure rate: 0.0 %.
To see unhandled errors, run with verbose=True.


In [54]:
set1[0].results['rate']['Calcite'].time


<xarray.DataArray 'time' (time: 26)>
array([1.00e-01, 1.00e+02, 2.00e+02, 3.00e+02, 4.00e+02, 5.00e+02, 2.00e+03,
       2.10e+03, 2.20e+03, 2.30e+03, 2.35e+03, 2.40e+03, 2.50e+03, 2.60e+03,
       2.70e+03, 2.80e+03, 2.90e+03, 3.00e+03, 4.00e+03, 5.00e+03, 6.00e+03,
       7.00e+03, 8.00e+03, 9.00e+03, 1.00e+04, 1.10e+04])
Coordinates:
  * time     (time) float64 0.1 100.0 200.0 300.0 ... 8e+03 9e+03 1e+04 1.1e+04

In [55]:
for i in range(6):
    print(set1[i].results['rate']['Calcite'].time[-1].to_numpy())

11000.0
9000.0
7000.0
6000.0
6000.0
5000.0


In [75]:
clays = ['Hsaponite(Mg)', 'Clinochlore', 'Chamosite(Daph', 'Celadonite']
basalt = ['Forsterite', 'Fayalite', 'Microcline', 'Ferrosilite(al', 'Enstatite(alph', 'Diopside', 'Anorthite', 'Albite(low)', 'Ilmenite']

def sum_vars(file, category, vars, name):
    """Sum variables in an Omphalos xarray"""
    sum_init = xr.zeros_like(file.results[category][vars[0]])
    file.results[category] = file.results[category].assign({name: sum_init})

    for var in vars:
        file.results[category][name] += file.results[category][var]

for file in set1:
    sum_vars(set1[file], 'volume', basalt, 'basalt')


In [86]:
%matplotlib notebook

import matplotlib as mpl
from matplotlib import pyplot as plt
from cmcrameri import cm
import numpy as np
mpl.rcParams.update(mpl.rcParamsDefault)

mpl.rcParams['axes.linewidth'] = 3
mpl.style.use('seaborn-poster')

time_steps = 25
i = 0
fontsize = 24

fig, axes = plt.subplots(1,1, figsize=(7,15))

category = 'saturation'
plot_var = 'Anhydrite'
file_num = 0

time = 20


j=0

#Prep fig before plotting
axes.xaxis.tick_top()
axes.xaxis.set_label_position('top')
axes.grid(False)
axes.tick_params(length=8, width=4)
if category == 'totcon':
    axes.set_xlabel(f'[{plot_var}] / M', fontweight='bold')
elif category == 'saturation':
    axes.set_xlabel(f'{plot_var} / $Omega$', fontweight='bold')
elif category == 'volume':
    axes.set_xlabel(f'{plot_var} / volume frac.', fontweight='bold')
elif category == 'pH':
    axes.set_xlabel(f'{plot_var}', fontweight='bold')
else:
    axes.set_xlabel(f'{plot_var} / UNITS NOT SPECD', fontweight='bold')
axes.set_ylabel('Depth / m', fontweight='bold')
#axis.set_ylim(0,750)
#axis.set_xlim(0,175)
for tick in axes.xaxis.get_major_ticks():
    tick.label2.set_fontweight('bold')
for tick in axes.yaxis.get_major_ticks():
    tick.label1.set_fontweight('bold')
axes.invert_yaxis()


# Line set loop (i.e. groups of lines sharing a property)
# Line instance loop

plot_indxs = [0,1,2,3,4,5]

for i in plot_indxs:
    try:
        #x = set1[i].results[category].isel(time=[12, 17, 18, 19, 20], Y=0, Z=0).mean('time')[plot_var].values
        x = set1[i].results[category].isel(time=time, Y=0, Z=0)[plot_var].values
        y = set1[i].results[category].isel(Y=0, Z=0).isel(time=time)['X'].values
    except IndexError:
        x=0
        y=0
    axes.plot(x, y, c=cm.batlow(i/len(plot_indxs)), label=f'{(i+1)*10} mM')
    axes.legend()

fig.tight_layout()

fig.savefig(f'{category}_{plot_var}_profile.png', dpi=300)
fig.show()

/var/folders/86/d5n0l2wj5yn0p84q_4jzm_6r0000gn/T/ipykernel_25038/2579911966.py:10: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  mpl.style.use('seaborn-poster')


<IPython.core.display.Javascript object>